# Mini Agent con Tools - Quanta

Se implementa un agente conversacional que puede:
1. Responder preguntas sobre el juego Quanta usando busqueda semantica en la KB
2. Buscar informacion general en Wikipedia
3. Citar sus fuentes de forma estructurada


In [1]:
# Instalacion de dependencias
%pip install -q \
  langchain langchain-core langchain-community \
  langchain-openai langchain-anthropic langchain-google-genai \
  langchain-text-splitters \
  requests pydantic sentence-transformers wikipedia

import os
import re
import json
import requests
import wikipedia
import time
from typing import List, Dict, Any

from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Providers
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings

Note: you may need to restart the kernel to use updated packages.


/Users/angiesanchez/Documents/GENAI_QUANTA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/angiesanchez/Documents/GENAI_QUANTA/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/angiesanchez/Documents/GENAI_QUANTA/.venv/lib/python3.9/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarnin

In [2]:
def get_llm():
    """
    Detecta automaticamente una API Key en el entorno y retorna el chat model correspondiente.
    Soporta: OPENAI_API_KEY, ANTHROPIC_API_KEY, GOOGLE_API_KEY.
    """
    if os.getenv("OPENAI_API_KEY"):
        return ChatOpenAI(model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"), temperature=0)
    if os.getenv("ANTHROPIC_API_KEY"):
        return ChatAnthropic(model=os.getenv("ANTHROPIC_MODEL", "claude-3-5-sonnet-latest"), temperature=0)
    if os.getenv("GOOGLE_API_KEY"):
        return ChatGoogleGenerativeAI(model=os.getenv("GOOGLE_MODEL", "gemini-1.5-pro"), temperature=0)
    
    raise RuntimeError("No se encontro una API Key valida en las variables de entorno.")

def get_embeddings():
    """
    Recomendamos usar embeddings locales para reducir latencia y costos.
    """
    if os.getenv("OPENAI_API_KEY"):
        emb_model = os.getenv("OPENAI_EMBEDDINGS_MODEL", "text-embedding-3-small")
        return OpenAIEmbeddings(model=emb_model)
    
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [3]:
KB_DOCS_RAW = [
    {
        "doc_id": "DOC_01",
        "title": "Quanta - Overview & Components",
        "text": """Quanta is a fictional strategy card game for 2-5 players (45-60 min).
Goal: earn the most Charge Points (CP) by building Sets, triggering Reactions, and controlling Zones.

Components:
- 90 Cards: Particle, Reactor, Catalyst, Event
- 50 Charge tokens (CP)
- 10 Zone tiles (A-J)
- 1 Round marker

Key terms:
- Set: 3+ cards sharing a symbol family
- Reaction: a combo that awards CP and may affect Zones
- Zone control: having the highest total Zone Power in a Zone"""
    },
    {
        "doc_id": "DOC_02",
        "title": "Quanta - Setup & Turn Structure",
        "text": """Setup:
1) Shuffle the deck. Deal 5 cards to each player.
2) Place 5 Zones face-up (random) in the center.
3) Each player starts with 3 CP.

Turn structure (in order):
A) Draw Phase: draw 1 card (2 if you control 2+ Zones).
B) Action Phase: take exactly 2 actions from:
   - Play a Particle (to your tableau)
   - Play a Catalyst (attach to a Particle)
   - Claim a Zone (commit cards as Zone Power)
   - Trigger a Reaction (if requirements met)
   - Swap 1 card from hand with top of discard (once per turn)
C) Cleanup: discard down to 7 cards."""
    },
    {
        "doc_id": "DOC_03",
        "title": "Quanta - Scoring & Endgame",
        "text": """You gain CP from:
- Completing Sets: +2 CP for a 3-card Set, +1 CP for each extra card beyond 3.
- Triggering Reactions: typically +3 CP (see Reaction rules); some Events modify CP.
- Zone Control: at end of each Round, +1 CP per Zone you control.

Endgame triggers when either:
- The Round marker reaches Round 8, OR
- A player reaches 25 CP immediately after scoring.

Tie-breakers:
1) Most Zones controlled
2) Most Reactions triggered
3) Fewest cards in hand"""
    },
    {
        "doc_id": "DOC_04",
        "title": "Quanta - Reactions, Events & Edge Cases",
        "text": """Reaction basics:
- A Reaction requires: (1) a Reactor card in your tableau AND (2) the listed Particle/Catalyst pattern.
- When triggered, discard the required cards, then apply the Reaction effect and gain CP.

Common edge cases:
- If two effects trigger at once, active player chooses order.
- Zone Control updates immediately when Zone Power changes.
- Events resolve instantly when played, then go to discard.
- You cannot trigger the same named Reaction more than once per turn.

Example Reaction:
Cascade: If you have Reactor + 2 Particles with matching family symbols, gain 3 CP, then draw 1."""
    },
    {
        "doc_id": "DOC_05",
        "title": "Quanta - Tournament Format (Optional Rules)",
        "text": """Tournament (best-of-3):
- Each game uses 5 Zones.
- Between games, players may swap 1 Zone tile (from reserve) into the pool.

Optional rules:
- Fast Mode: endgame at Round 6 instead of Round 8.
- Draft Start: deal 7 cards, keep 5, pass 2 left, repeat once.

Dispute rule:
If players disagree on an interaction, pause and consult the rules.
If still unclear, resolve in the least game-breaking way and note it for later."""
    },
]

In [4]:
# Procesamiento e indexacion de documents

# Convertir raw documents a objetos Document de LangChain
documents = [
    Document(
        page_content=doc["text"],
        metadata={
            "doc_id": doc["doc_id"],
            "title": doc["title"],
            "source": "Quanta KB"
        }
    )
    for doc in KB_DOCS_RAW
]

# Dividir en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

split_docs = text_splitter.split_documents(documents)

# Embeddings e indexar 
embeddings = get_embeddings()
vector_store = InMemoryVectorStore(embeddings)

# Agregar documentos al vector store
vector_store.add_documents(split_docs)

print(f"Indexados {len(split_docs)} chunks de {len(documents)} documentos")
print(f"Vector store inicializado con embeddings: {type(embeddings).__name__}")

Indexados 7 chunks de 5 documentos
Vector store inicializado con embeddings: OpenAIEmbeddings


In [5]:
# Implementacion de las tools

@tool
def kb_search(query: str, k: int = 3) -> List[Dict[str, Any]]:
    """
    Knowledge Base Search Tool:
    Busca información en el manual de reglas de Quanta. 
    Retorna los k documentos más relevantes con su contenido y metadata.
    Debe usarse para cualquier duda sobre mecánicas, fases, puntos o torneos.
    """
    # Implementa la búsqueda sobre el vector_store y retorna los resultados con metadata para citas.


    results = vector_store.similarity_search(query, k=k)
    
    formatted_results = [
        {
            "content": doc.page_content,
            "doc_id": doc.metadata.get("doc_id", "UNKNOWN"),
            "title": doc.metadata.get("title", "Untitled")
        }
        for doc in results
    ]
    
    return formatted_results


@tool
def wikipedia_search(query: str, sentences: int = 3) -> Dict[str, Any]:
    """
    Wikipedia Search Tool:
    Busca información en Wikipedia en español.
    Retorna un resumen breve, el título y la URL de la página.
    Debe usarse como herramienta para obtener información general o contexto adicional.
    """

    try:
        
        wikipedia.set_lang("es")
        
        try:
            page = wikipedia.page(query, auto_suggest=True)
            summary = wikipedia.summary(query, sentences=sentences, auto_suggest=True)
            
            return {
                "summary": summary,
                "url": page.url,
                "title": page.title
            }
        except wikipedia.exceptions.DisambiguationError as e:
            page = wikipedia.page(e.options[0])
            summary = wikipedia.summary(e.options[0], sentences=sentences)
            
            return {
                "summary": summary,
                "url": page.url,
                "title": page.title,
                "note": f"Se encontraron multiples opciones. Mostrando: {page.title}"
            }
        except wikipedia.exceptions.PageError:
            return {
                "summary": f"No se encontro informacion sobre '{query}' en Wikipedia.",
                "url": None,
                "title": None,
                "error": "Page not found"
            }
    except Exception as e:
        return {
            "summary": f"Error al buscar en Wikipedia: {str(e)}",
            "url": None,
            "title": None,
            "error": str(e)
        }


tools = [kb_search, wikipedia_search]

print("Tools definidas")

Tools definidas


In [6]:
# Definicion del prompt del sistema y creacion del agente

SYSTEM_PROMPT = """Eres un asistente experto en el juego de cartas Quanta y en informacion general.

REGLAS DE RUTEO (MUY IMPORTANTE):
1. Para CUALQUIER pregunta sobre el juego Quanta (reglas, mecanicas, componentes, fases, puntos, torneos):
   -> SIEMPRE usa la herramienta 'kb_search'
   
2. Para preguntas sobre informacion general, personas, conceptos del mundo real:
   -> USA la herramienta 'wikipedia_search'

3. Si no estas seguro, usa kb_search primero para verificar si es sobre Quanta

FORMATO DE RESPUESTA OBLIGATORIO:
1. Responde la pregunta de forma clara y concisa
2. SIEMPRE incluye al final una seccion "Fuentes:" con las citas
3. Para kb_search: cita como "[DOC_XX - Titulo]"
4. Para wikipedia_search: cita como "[Wikipedia: Titulo (URL)]"

IMPORTANTE:
- Se preciso y fundamenta tus respuestas en las herramientas
- Si la informacion no esta disponible, indicalo claramente
- Manten un tono amigable pero profesional
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

llm = get_llm()
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print(f"Agente creado con LLM: {type(llm).__name__}")
print(f"Herramientas disponibles: {[t.name for t in tools]}")

Agente creado con LLM: ChatOpenAI
Herramientas disponibles: ['kb_search', 'wikipedia_search']


In [7]:
# Funcion de prueba con formato de salida estructurado

def run_query(query: str):
    """
    Ejecuta una consulta y muestra la respuesta con formato estructurado.
    """
    print(f"CONSULTA: {query}")
    

    # Iniciar tracking
    start_time = time.time()
    result = agent_executor.invoke({"input": query})
    total_time = time.time() - start_time

    final_answer = result["output"]
    intermediate_steps = result.get("intermediate_steps", [])
    
    print("\nTRACE DE HERRAMIENTAS:")
    if intermediate_steps:
        for i, (action, observation) in enumerate(intermediate_steps, 1):
            tool_name = action.tool
            tool_input = action.tool_input
            
            print(f"  {i}. Herramienta: {tool_name}")
            print(f"     Input: {tool_input}")
            
            if tool_name == "kb_search":
                print(f"     Razon: Buscando informacion sobre Quanta en la base de conocimiento")
            elif tool_name == "wikipedia_search":
                print(f"     Razon: Buscando informacion general en Wikipedia")
            print()
    else:
        print("  (No se usaron herramientas)")
    
    print("\nRESPUESTA:")
    print(final_answer)
    
    if "Fuentes" not in final_answer:
        print("\nFUENTES:")
        
        sources_found = False
        
        for action, observation in intermediate_steps:
            if action.tool == "kb_search" and observation:
                try:
                    if isinstance(observation, list):
                        for item in observation:
                            if isinstance(item, dict):
                                doc_id = item.get('doc_id', 'UNKNOWN')
                                title = item.get('title', 'Untitled')
                                print(f"  - [{doc_id} - {title}]")
                                sources_found = True
                except:
                    pass
            
            elif action.tool == "wikipedia_search" and observation:
                try:
                    if isinstance(observation, dict):
                        title = observation.get('title')
                        url = observation.get('url')
                        if title and url:
                            print(f"  - [Wikipedia: {title}]")
                            print(f"    URL: {url}")
                            sources_found = True
                except:
                    pass
        
        if not sources_found:
            print("  (Ver fuentes en la respuesta del agente)")
    
    print(f"\n\nTiempo total:   {total_time:.3f}s")
    print(f"Tools usadas:    {len(intermediate_steps)}")
    # Contar resultados
    num_results = 0
    for action, observation in intermediate_steps:
        if isinstance(observation, list):
            num_results += len(observation)
        elif isinstance(observation, dict):
            num_results += 1
        
    print(f"Resultados recuperados: {num_results}")

print("CASOS DE PRUEBA LISTOS")

CASOS DE PRUEBA LISTOS


## Sección de Pruebas

In [8]:
# Test 1
run_query("En Quanta, ¿cómo se dispara una Reaction y qué pasa con los Catalyst usados como costo?")

CONSULTA: En Quanta, ¿cómo se dispara una Reaction y qué pasa con los Catalyst usados como costo?


> Entering new AgentExecutor chain...

Invoking: `kb_search` with `{'query': 'Reaction Catalyst Quanta'}`


[{'content': 'Reaction basics:\n- A Reaction requires: (1) a Reactor card in your tableau AND (2) the listed Particle/Catalyst pattern.\n- When triggered, discard the required cards, then apply the Reaction effect and gain CP.\n\nCommon edge cases:\n- If two effects trigger at once, active player chooses order.\n- Zone Control updates immediately when Zone Power changes.\n- Events resolve instantly when played, then go to discard.\n- You cannot trigger the same named Reaction more than once per turn.', 'doc_id': 'DOC_04', 'title': 'Quanta - Reactions, Events & Edge Cases'}, {'content': 'Example Reaction:\nCascade: If you have Reactor + 2 Particles with matching family symbols, gain 3 CP, then draw 1.', 'doc_id': 'DOC_04', 'title': 'Quanta - Reactions, Events & Edge Cases'}, {'conte

In [9]:
# test 2
run_query("Enumera las formas principales de obtener Charge Points (CP) en Quanta.")

CONSULTA: Enumera las formas principales de obtener Charge Points (CP) en Quanta.


> Entering new AgentExecutor chain...

Invoking: `kb_search` with `{'query': 'Charge Points (CP)'}`


[{'content': 'Quanta is a fictional strategy card game for 2-5 players (45-60 min).\nGoal: earn the most Charge Points (CP) by building Sets, triggering Reactions, and controlling Zones.\n\nComponents:\n- 90 Cards: Particle, Reactor, Catalyst, Event\n- 50 Charge tokens (CP)\n- 10 Zone tiles (A-J)\n- 1 Round marker\n\nKey terms:\n- Set: 3+ cards sharing a symbol family\n- Reaction: a combo that awards CP and may affect Zones\n- Zone control: having the highest total Zone Power in a Zone', 'doc_id': 'DOC_01', 'title': 'Quanta - Overview & Components'}, {'content': 'You gain CP from:\n- Completing Sets: +2 CP for a 3-card Set, +1 CP for each extra card beyond 3.\n- Triggering Reactions: typically +3 CP (see Reaction rules); some Events modify CP.\n- Zone Control: at end of each Round, +1 CP per Zone you co

In [10]:
# Test 3
run_query("En el formato de torneo de Quanta, ¿qué puedes cambiar entre partidas?")

CONSULTA: En el formato de torneo de Quanta, ¿qué puedes cambiar entre partidas?


> Entering new AgentExecutor chain...

Invoking: `kb_search` with `{'query': 'formato de torneo Quanta cambios entre partidas'}`


[{'content': 'Tournament (best-of-3):\n- Each game uses 5 Zones.\n- Between games, players may swap 1 Zone tile (from reserve) into the pool.\n\nOptional rules:\n- Fast Mode: endgame at Round 6 instead of Round 8.\n- Draft Start: deal 7 cards, keep 5, pass 2 left, repeat once.\n\nDispute rule:\nIf players disagree on an interaction, pause and consult the rules.\nIf still unclear, resolve in the least game-breaking way and note it for later.', 'doc_id': 'DOC_05', 'title': 'Quanta - Tournament Format (Optional Rules)'}, {'content': 'Setup:\n1) Shuffle the deck. Deal 5 cards to each player.\n2) Place 5 Zones face-up (random) in the center.\n3) Each player starts with 3 CP.', 'doc_id': 'DOC_02', 'title': 'Quanta - Setup & Turn Structure'}, {'content': 'Turn structure (in order):\

In [11]:
# Test 4
run_query("¿Quién fue Ada Lovelace? ")

CONSULTA: ¿Quién fue Ada Lovelace? 


> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `{'query': 'Ada Lovelace', 'sentences': 3}`


{'summary': 'Augusta Ada King, condesa de Lovelace (Londres, 10 de diciembre de 1815-íd., 27 de noviembre de 1852), registrada al nacer como Augusta Ada Byron y conocida habitualmente como Ada Lovelace, fue una matemática y escritora británica, célebre sobre todo por su trabajo acerca de la computadora mecánica de uso general de Charles Babbage, la denominada máquina analítica. Fue la primera en reconocer que la máquina tenía aplicaciones más allá del cálculo puro y en haber publicado lo que se reconoce hoy como el primer algoritmo destinado a ser procesado por una máquina, por lo que se le considera como la primera programadora de ordenadores, aunque no existen pruebas documentales de tales afirmaciones (véase la sección Polémica sobre sus contribuciones).\u200b\u200b\u200b\nLovelace fue la única hija legítima del poeta Lord Byron

In [12]:
# Otras consultas
run_query("Quien es angie sanchez marquina")

CONSULTA: Quien es angie sanchez marquina


> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `{'query': 'Angie Sanchez Marquina'}`


{'summary': "No se encontro informacion sobre 'Angie Sanchez Marquina' en Wikipedia.", 'url': None, 'title': None, 'error': 'Page not found'}No se encontró información sobre "Angie Sanchez Marquina" en Wikipedia. Es posible que no sea una figura pública ampliamente reconocida o que no haya información disponible en esa fuente. Si tienes más contexto o detalles sobre quién es, podría ayudarte mejor. 

Fuentes: [Wikipedia: Página no encontrada]

> Finished chain.

TRACE DE HERRAMIENTAS:
  1. Herramienta: wikipedia_search
     Input: {'query': 'Angie Sanchez Marquina'}
     Razon: Buscando informacion general en Wikipedia


RESPUESTA:
No se encontró información sobre "Angie Sanchez Marquina" en Wikipedia. Es posible que no sea una figura pública ampliamente reconocida o que no haya información disponible en esa fuente. Si tienes más c

In [13]:
# Otras consultas
run_query("Compara Quanta con ajedrez")

CONSULTA: Compara Quanta con ajedrez


> Entering new AgentExecutor chain...

Invoking: `kb_search` with `{'query': 'Quanta'}`


[{'content': 'Quanta is a fictional strategy card game for 2-5 players (45-60 min).\nGoal: earn the most Charge Points (CP) by building Sets, triggering Reactions, and controlling Zones.\n\nComponents:\n- 90 Cards: Particle, Reactor, Catalyst, Event\n- 50 Charge tokens (CP)\n- 10 Zone tiles (A-J)\n- 1 Round marker\n\nKey terms:\n- Set: 3+ cards sharing a symbol family\n- Reaction: a combo that awards CP and may affect Zones\n- Zone control: having the highest total Zone Power in a Zone', 'doc_id': 'DOC_01', 'title': 'Quanta - Overview & Components'}, {'content': 'You gain CP from:\n- Completing Sets: +2 CP for a 3-card Set, +1 CP for each extra card beyond 3.\n- Triggering Reactions: typically +3 CP (see Reaction rules); some Events modify CP.\n- Zone Control: at end of each Round, +1 CP per Zone you control.\n\nEndgame triggers when either:\n- The Round mark